# How Fast Can We Go?

## This notebook demonstrates relative speed differences between various coding approaches for a simple find and replace operation. Approaches compared are numpy.where(), vectorized Python, for loops, and list comprehensions.

### Set up the tests

In [1]:
import numpy as np

Create an array of 100 million 1's and 0's

In [2]:
largedata = np.random.randint(0,2,100000000)
np.unique(largedata)

array([0, 1])

Create an array of just 100 1's and 0's

In [3]:
smalldata = np.random.randint(0,2,100)
np.unique(smalldata)

array([0, 1])

## numpy.where vs. vectorized approach on largedata

Use NumPy function (np.where) to change all 0's to -1's on largedata and time it.

In [4]:
%time largedata = np.where(largedata < 1, -1, 1)

CPU times: user 157 ms, sys: 157 ms, total: 314 ms
Wall time: 309 ms


* On my laptop, this usually averaged around 300 milliseconds

Confirm array changed as expected

In [5]:
np.unique(largedata)

array([-1,  1])

Create a vectorized action to change the -1's to 0 and time it

In [6]:
%time largedata[largedata == -1] = 0

CPU times: user 566 ms, sys: 4 ms, total: 570 ms
Wall time: 566 ms


* On my laptop, this usually averaged around 580 milliseconds

Confirm list was changed

In [7]:
np.unique(largedata)

array([0, 1])

## numpy.where vs. vectorized approach on smalldata

Now compare the approaches on smalldata

In [12]:
%time smalldata = np.where(smalldata < 1, -1, 1)

CPU times: user 100 µs, sys: 2 µs, total: 102 µs
Wall time: 109 µs


* On my laptop, this usually averaged around 110 microseconds

In [13]:
%time smalldata[smalldata == -1] = 0

CPU times: user 75 µs, sys: 0 ns, total: 75 µs
Wall time: 80.1 µs


* On my laptop, this usually averaged around 80 microseconds

### Interesting side note: The NumPy advantage here goes away as data gets smaller, and in fact when I tested the array size of 100 instead of 100 million, the vectorized approach was *faster* than the NumPy approach.

## loops and list comprehensions on largedata

Try the really slow way. Define a function that uses a for loop and time its execution

In [14]:
def slowloop(data):
    for i in range(0,len(data)):
        if data[i] < 1:
            data[i] = -1
    return data

In [15]:
%time largedata = slowloop(largedata)

CPU times: user 23 s, sys: 40.9 ms, total: 23 s
Wall time: 23 s


* On my laptop, this usually averaged around 21 seconds

In [16]:
np.unique(largedata)

array([-1,  1])

Create a list comprehension to change all 0's back to -1 and see how fast it runs

In [17]:
%time largedata = [i if i > 0 else 0 for i in largedata]

CPU times: user 16.3 s, sys: 524 ms, total: 16.8 s
Wall time: 16.8 s


* On my laptop, this usually averaged around 17 seconds

Confirm array is changed

In [18]:
np.unique(largedata)

array([0, 1])

### The for loop and list comprehension approaches are both dramatically slower (30-80x slower) than the numpy.where and vectorized approaches. List comprehension working on a numpy array was about 20% faster than native for loop. However...

Notice that the list comprehension, as the name would imply, converts the array to a list

In [19]:
type(largedata)

list

Now notice the change in performance for slow loops on a list object vs. an array

In [20]:
%time largedata = slowloop(largedata)

CPU times: user 11.2 s, sys: 303 ms, total: 11.5 s
Wall time: 11.5 s


* On my laptop, this usually averaged around 11 seconds

### If the for loop approach starts with a list vs a NumPy array, the time the operation takes is cut in half. Fascinating.

In [21]:
np.unique(largedata)

array([-1,  1])

Does starting with a list also improve the list comprehension performance?

In [22]:
%time largedata = [i if i > 0 else 0 for i in largedata]

CPU times: user 8.47 s, sys: 262 ms, total: 8.73 s
Wall time: 8.73 s


### In both looping scenarios, starting with a list reduced the time to process by approximately 50%. 

# Final Analysis

## On data of any size, using a NumPy function (if one exists that can perform your operation) is going to be faster than any native Python approach.

## Vectorized Python smokes any loop-based approach in terms of speed.

## List comprehensions outperform native for loops by about 20%, but are still many multiple times slower than vectorized Python.

## If using a list comprehension or a for loop, formatting your data as a list will improve performance by approximately 50% vs. using a NumPy array.